### RFM 분석 실습
- Recency: 얼마나 최근에 구매했는가
- Frequency: 얼마나 자주 구매했는가
- Monetary: 얼마나 많은 금액을 지출했는가

### 고객 분석

> voice_no: 송장 번호. 명사 같은. 각 작업에 고유하게 할당된 문자 'I'와 6자리 정수의 조합  
customer_id: 고객번호. 명사 같은. 각 작업에 고유하게 할당된 문자 'C'와 6자리 정수의 조합  
gender: 고객 성별의 문자a열 변수  
age: 고객 연령의 양의 정수 변수  
category: 구매한 상품 카테고리의 문자열 변수 
quantity: 거래당 각 제품(항목)의 수량  
price: 단가. 숫자. 단위당 제품 가격은 터키 리라(TL)  
payment_method: 거래에 사용된 결제 방법(현금, 신용 카드 또는 직불 카드)의 문자열 변수  
invoice_date: 송장 날짜. 거래가 발생한 날  
shopping_mall: 거래가 이루어진 쇼핑몰 이름의 문자열 변수  

### Ⅰ. 데이터 탐색

In [1]:
import pandas as pd

cs_df = pd.read_csv("./datasets/customer_shopping_data.csv")
display(cs_df)

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.65,Credit Card,21/09/2022,Kanyon
99453,I325143,C569580,Male,27,Food & Beverage,2,10.46,Cash,22/09/2021,Forum Istanbul
99454,I824010,C103292,Male,63,Food & Beverage,2,10.46,Debit Card,28/03/2021,Metrocity
99455,I702964,C800631,Male,56,Technology,4,4200.00,Cash,16/03/2021,Istinye Park


#### ✏️ 기획 
1. 쇼핑몰 별 이용 빈도 분석(인기순,,,)
2. 쇼핑몰 카테고리.. 분석 뭐 파는 곳인지? > 그래서 유사한 거 끼리 분석할 수 있을듯
4. 쇼핑목 별 주 사용자 연령층, 성별 분석
5. 쇼핑몰 별..rfm해야될듯 전체에서 하는ㄴ 거 아니고 쇼핑몰 인원끼리 모아서 총합하고 구해야됨!!!(최근 이용 날짜는 평균으로 구할 수 있지 않을까?)
6. 매출 분석

#### 📊 데이터 정보 확인

In [7]:
# cs_df 의 정보 가져오기
cs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_no      99457 non-null  object 
 1   customer_id     99457 non-null  object 
 2   gender          99457 non-null  object 
 3   age             99457 non-null  int64  
 4   category        99457 non-null  object 
 5   quantity        99457 non-null  int64  
 6   price           99457 non-null  float64
 7   payment_method  99457 non-null  object 
 8   invoice_date    99457 non-null  object 
 9   shopping_mall   99457 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB


In [15]:
# 상품 카테고리 확인
cs_df['category'].value_counts()

category
Clothing           34487
Cosmetics          15097
Food & Beverage    14776
Toys               10087
Shoes              10034
Souvenir            4999
Technology          4996
Books               4981
Name: count, dtype: int64

In [17]:
# 결제 방식 확인
cs_df['payment_method'].value_counts()

payment_method
Cash           44447
Credit Card    34931
Debit Card     20079
Name: count, dtype: int64

#### 📊 결측치 검사

In [10]:
# cs_df의 결측치 개수를 가져오기
cs_df.isna().sum()

invoice_no        0
customer_id       0
gender            0
age               0
category          0
quantity          0
price             0
payment_method    0
invoice_date      0
shopping_mall     0
dtype: int64

#### 📊 중복행 검사

In [11]:
# cs_df의 중복행 개수를 가져오기
cs_df.duplicated().sum()

0

#### 💡 분석
- 쇼핑몰 별 거래 데이터 프레임의 전체 정보
- 10개 컬럼이 존재함.
- 99457개 데이터 존재함.
- 결측치가 존재하지 않음.
- 중복 데이터가 존재하지 않음.

#### ❓ 가설
- 환불된 주문 데이터가 있을까?

#### 📊 숫자형 데이터 개수, 평균, 표준편차, 최소값, 사분위 분포도, 최대값 확인

In [6]:
cs_df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,99457.0,43.427089,14.990054,18.00,30.00,43.0,56.00,69.0
quantity,99457.0,3.003429,1.413025,1.00,2.00,3.0,4.00,5.0
price,99457.0,689.256321,941.184567,5.23,45.45,203.3,1200.32,5250.0


#### 💡 분석
- 고객의 평균 연령은 약 43살으로 나타남.
- 최고 연령은 69살이며, 최소 연령은 18으로 나타남.
- 고객 당 평균 약 1.4개의 제품을 구매한 것으로 나타남.
- 상품 단가는 평균 약 689.427로 나타남.
- 상품의 최고 단가는 5250이며, 최소 단가는 5.23으로 나타남.

#### 📌 시사점
- 상품 최소 단가가 음수로 나오지 않았기 때문에 환불된 데이터가 없는 것으로 판단함.
- 나이, 개수는 이상치 제거가 적절하지 않다고 판단되므로, 별도의 이상치 제거는 실시하지 않음.

#### ❓ 가설
- 주문번호 중 I로 시작하지 않는 주문번호가 있을까?
- 고객번호 중 C로 시작하지 않는 고객번호가 있을까?